<a href="https://colab.research.google.com/github/marekk1717/letsautomate/blob/master/Commvault_Python_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## **Installation of cvpysdk and required dependencies**

In [0]:
!pip install xmltodict future requests beautifultable
!pip install git+https://github.com/CommvaultEngg/cvpysdk.git@dev

In [0]:
from cvpysdk.commcell import Commcell
import getpass
import json
from beautifultable import BeautifulTable
import pandas as pd
import random



---
<br><br>
## **Connect to WebConsole**

In [17]:
WEBCONSOLE_URL = input('WebConsole URL: ')
COMMCELL_USERNAME = input('Username: ')
COMMCELL_PASSWORD = getpass.getpass(prompt='Password: ')

commcell = Commcell(webconsole_hostname=WEBCONSOLE_URL, commcell_username=COMMCELL_USERNAME, commcell_password=COMMCELL_PASSWORD)
print('Connected to ' + commcell.commserv_hostname + ', Version: v11SP' + str(commcell.commserv_version))

Connected to COMMSERVE.cv.lab, Version: v11SP18




---
<br><br>

## **List of active backup jobs**

In [57]:
active_backup_jobs = commcell.job_controller.active_jobs(limit=1000, job_filter='Backup', job_summary='Full')
table = BeautifulTable(max_width=300)
table.column_headers = ["JOB ID", "Client", "AppType", "% Complete", 'AppSize (GB)']
for job in active_backup_jobs:
  table.append_row([job, active_backup_jobs[job]['destinationClient']['clientName'], active_backup_jobs[job]['appTypeName'], 
                    active_backup_jobs[job]['percentComplete'], active_backup_jobs[job]['sizeOfApplication']/1024/1024/1024])
print(table)

+--------+----------------------------+------------------+------------+--------------+
| JOB ID |           Client           |     AppType      | % Complete | AppSize (GB) |
+--------+----------------------------+------------------+------------+--------------+
| 300072 |       FileServerData       |   File System    |     25     |     0.0      |
+--------+----------------------------+------------------+------------+--------------+
| 300053 |         exchange2          |   File System    |     34     |    3.234     |
+--------+----------------------------+------------------+------------+--------------+
| 300015 |       Vcenter_OnPrem       |  Virtual Server  |     0      |     0.0      |
+--------+----------------------------+------------------+------------+--------------+
| 299999 |          vmlinux           |   File System    |     0      |     0.0      |
+--------+----------------------------+------------------+------------+--------------+
| 299996 |       ExchangeHybrid       | Exc

---
<br><br>
## **Completed backup jobs (1h)**

In [58]:
completed_backup_jobs = commcell.job_controller.finished_jobs(limit=1000, job_filter='Backup', job_summary='Full', lookup_time=1)
table = BeautifulTable(max_width=300)
table.column_headers = ["JOB ID", "Client", "AppType", 'AppSize (GB)', 'Media Size (GB)', 'Status']
for job in completed_backup_jobs:
  table.append_row([job, completed_backup_jobs[job]['destinationClient']['clientName'], completed_backup_jobs[job]['appTypeName'], 
                    completed_backup_jobs[job]['sizeOfApplication']/1024/1024/1024, completed_backup_jobs[job]['sizeOfMediaOnDisk']/1024/1024/1024, 
                    completed_backup_jobs[job]['status']])
print(table)

+--------+-----------------+------------------+--------------+-----------------+-----------------+
| JOB ID |     Client      |     AppType      | AppSize (GB) | Media Size (GB) |     Status      |
+--------+-----------------+------------------+--------------+-----------------+-----------------+
| 300071 |     search      |   File System    |    0.222     |      0.009      |    Completed    |
+--------+-----------------+------------------+--------------+-----------------+-----------------+
| 300070 |    commserve    |   File System    |     0.0      |       0.0       |    Completed    |
+--------+-----------------+------------------+--------------+-----------------+-----------------+
| 300069 |    commserve    | Active Directory |     0.0      |      0.001      |    Completed    |
+--------+-----------------+------------------+--------------+-----------------+-----------------+
| 300068 |  CVDemo-Gdrive  |    Cloud Apps    |     0.0      |       0.0       |    Completed    |
+--------+

0.25.3




---
<br><br>

## **Jobs stats (last 24h)**

In [91]:
completed_backup_jobs = commcell.job_controller.finished_jobs(limit=1000, job_filter='Backup', job_summary='Full', lookup_time=24)
df = pd.DataFrame(columns=['jobid', 'client', 'app_type', 'app_size', 'media_size', 'status'])
a = 0
for job in completed_backup_jobs:
  df.loc[a] = [job, completed_backup_jobs[job]['destinationClient']['clientName'], completed_backup_jobs[job]['appTypeName'], 
                    completed_backup_jobs[job]['sizeOfApplication']/1024/1024/1024, completed_backup_jobs[job]['sizeOfMediaOnDisk']/1024/1024/1024, 
                    completed_backup_jobs[job]['status']]
  a+=1
df_stats = df[['status','jobid']].groupby(['status']).count().reset_index()
df_stats.columns = ['Status','Jobs']
df_stats

,Status,Jobs
0,Completed,315
1,Completed w/ one or more errors,25
2,Failed,21
3,Failed to Start,53
4,Killed,4




---

<br><br>
## **Media Size per Client (Top 10, last 24h)** 

In [94]:
df_stats = df[['client','media_size']].groupby(['client']).sum().reset_index().sort_values(by=['media_size'], ascending=False)
df_stats.columns = ['Client','Media Size (GB)']
df_stats.head(10)

,Client,Media Size (GB)
34,Vcenter_OnPrem,6.133442
35,awsma,4.655011
36,azma1,3.386943
38,exchange2,2.947183
5,Azure,2.223956
17,FileServerData,1.253389
41,hs02.cv.lab,1.143417
4,Amazon,1.142256
40,hs01.cv.lab,1.126840
42,hs03.cv.lab,1.116640




---
<br><br>

##**Media Size per Agent (last 24h)**

In [93]:
df_stats = df[['app_type','media_size']].groupby(['app_type']).sum().reset_index().sort_values(by=['media_size'], ascending=False)
df_stats.columns = ['Application Type','Media Size (GB)']
df_stats

,Application Type,Media Size (GB)
5,File System,15.796970
12,Virtual Server,9.499654
1,Big Data Apps,0.674927
11,Sharepoint Server,0.313044
10,SQL Server,0.023627
2,Cloud Apps,0.015136
3,Exchange Database,0.012905
0,Active Directory,0.009162
4,Exchange Mailbox,0.008621
7,NDMP,0.001020




---
<br><br>
# **Properties of randomly selected client**


In [127]:
cls = list(commcell.clients.all_clients.keys())
cltmp = commcell.clients.get(cls[random.randrange(0, len(cls))])
table = BeautifulTable(max_width=300)
table.column_headers = ["Client Name", "Hostname", "Install Dir", 'Data Management Enabled', 'Recovery Enabled', 'Version', 'Service Pack', 'Installed Agents']
table.append_row([cltmp.display_name, cltmp.client_hostname, cltmp.install_directory, cltmp.is_data_management_enabled, cltmp.is_data_recovery_enabled, cltmp.version, cltmp.service_pack, list(cltmp.agents.all_agents.keys())])
print(table)


+-------------+---------------+-----------------------------------------+-------------------------+------------------+---------+--------------+-------------------------------------------------------------------------------+
| Client Name |   Hostname    |               Install Dir               | Data Management Enabled | Recovery Enabled | Version | Service Pack |                               Installed Agents                                |
+-------------+---------------+-----------------------------------------+-------------------------+------------------+---------+--------------+-------------------------------------------------------------------------------+
|   search    | SEARCH.cv.lab | C:\Program Files\Commvault\ContentStore |          True           |       True       |   11    |      18      | ['continuous data replicator', 'sql server', 'virtual server', 'file system'] |
+-------------+---------------+-----------------------------------------+-------------------------+-----



---
## **List of Plans**


In [154]:
table = BeautifulTable(max_width=300)
table.column_headers = ["Plan Name", "RPO(h)"]
for plan in commcell.plans.all_plans:
  plantmp = commcell.plans.get(plan)
  table.append_row([plan, plantmp.sla_in_minutes/60])
print(table)

  


+-----------------------------------------------+--------+
|                   Plan Name                   | RPO(h) |
+-----------------------------------------------+--------+
|           basic_data_30d_hyperscale           |  3.0   |
+-----------------------------------------------+--------+
|           basic_virtual_servers_30d           |  3.0   |
+-----------------------------------------------+--------+
|                  awsbase_30d                  |  3.0   |
+-----------------------------------------------+--------+
|                 azurebase_30d                 |  3.0   |
+-----------------------------------------------+--------+
|                 aws databases                 |  4.0   |
+-----------------------------------------------+--------+
|                onpremsnap plan                |  4.0   |
+-----------------------------------------------+--------+
|           cvdemo user mailbox plan            |  0.0   |
+-----------------------------------------------+-------



---
<br><br>
## **List of Tenants (Companies)**


In [0]:
a = commcell.storage_pools.get('az_east_global_dedupe_policy')

In [111]:
table = BeautifulTable(max_width=300)
table.column_headers = ["Tenant Name", "Assigned Plans", "Number of Servers", 'Auto Discovery Enabled', 'Auth Code Enabled', 'Tenant Admin']

for org in commcell.organizations.all_organizations:
  orgtmp = commcell.organizations.get(org)
  table.append_row([org, orgtmp.plans, orgtmp.machine_count, orgtmp.is_auto_discover_enabled, orgtmp.is_auth_code_enabled, orgtmp.contacts])
print(table)

+--------------------+--------------------+-------------------+------------------------+-------------------+----------------------------+
|    Tenant Name     |   Assigned Plans   | Number of Servers | Auto Discovery Enabled | Auth Code Enabled |        Tenant Admin        |
+--------------------+--------------------+-------------------+------------------------+-------------------+----------------------------+
| gnoogle demolition |         []         |         3         |         False          |       True        | ['Gnoogle\\company1admin'] |
+--------------------+--------------------+-------------------+------------------------+-------------------+----------------------------+
|  cvfieldsecompany  | ['cvfieldcompany'] |         2         |         False          |       True        |    ['CVfield\\bgates']     |
+--------------------+--------------------+-------------------+------------------------+-------------------+----------------------------+
|       enron        |         [] 